# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [2]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras import Input, Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, LSTM
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

Using TensorFlow backend.


### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3866368574998311964
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4828102656
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17810517838333322114
physical_device_desc: "device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [4]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [5]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les Ã©tats-unis est gÃ©nÃ©ralement froid en juillet , et il gÃ¨le habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [6]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [7]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    tokenizer = Tokenizer(split=' ', char_level=False)
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer           # texts_to_sequences_generator(), Yields individual sequences.

tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [8]:
text_tokenized

[[1, 2, 4, 5, 6, 7, 1, 8, 9],
 [10, 11, 12, 2, 13, 14, 15, 16, 3, 17],
 [18, 19, 3, 20, 21]]

In [9]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    if length is None:
        length = max([len(sentence) for sentence in x])
    
    return pad_sequences(x, maxlen=length, padding='post', truncating='post')

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [10]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 345


In [11]:
print(type(preproc_english_sentences))
print(preproc_english_sentences.shape)

<class 'numpy.ndarray'>
(137861, 15)


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [12]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1) if index_to_words[prediction]!='<PAD>'] )

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

#### Inputs/Outputs shape Basic RNN

- LSTM or GRU Input shape (batchsize, timesteps, 1). The timesteps is the length of the input sequences (15 for en, 21 for fr)
- output shape LSTM (batchsize, timesteps, num_units) because Return_sequence activated to return the output at each timestep
- Use of TimeDistributed wrapper over Dense(unitsoutput=target_vocab_size) to process each timestep
- for each time step, Dense outputs a probability distribution over target_vocab to predict next word

In [13]:
def token_to_words(sequence, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return [index_to_words[token] for token in sequence if index_to_words[token]!='<PAD>']

In [56]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    learning_rate = 0.001
    english_input = Input(shape=input_shape[1:], name="input_layer")    # the shape is (input length x 1) as batchsize excluded
    
    # LSTM takes as input (batchsize,input_length,1) and outputs (batchsize, input_length, 64) because return-seq=True
    x = LSTM(64, return_sequences=True, activation="tanh", name="LSTM_layer")(english_input)
    preds = TimeDistributed(Dense(french_vocab_size, activation="softmax"), name="Dense_layer")(x)
    model = Model(inputs=english_input, outputs=preds, name='simple_LSTM')
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))  # reshape as 3D (batchsize, timesteps, 1) for LSTM input

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

simple_rnn_model.summary()

simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     (None, 21, 1)             0         
_________________________________________________________________
LSTM_layer (LSTM)            (None, 21, 64)            16896     
_________________________________________________________________
Dense_layer (TimeDistributed (None, 21, 345)           22425     
Total params: 39,321
Trainable params: 39,321
Non-trainable params: 0
_________________________________________________________________
Train on 110288 samples, validate on 27573 samples
Epoch 1/20
110288/110288 [==============================] - 9s 84us/step - loss: 3.4196 - acc: 0.4184 - val_loss: 2.5644 - val_acc: 0.4605
Epoch 2/20
110288/110288 [==============================] - 8s 69us/step - loss: 2.4462 - acc: 0.4693 - val_loss: 2.3099 - val_acc: 0.4826
Epoch 3/20
110288/110288 [==============================] - 8s 69us/step 

In [14]:
def translate(prediction, gold_standard):
    
    translation = logits_to_text(prediction[0], french_tokenizer)
    standard = ' '.join(token_to_words(gold_standard[0][:,0],french_tokenizer)) 
    print('---- Gold standard ----')
    print(standard)
    print()
    print('---- Prediction ----')
    for w_t, w_s in zip(translation.split(), standard.split()):
        if w_t == w_s:
            print('\033[0;30;0m','{}'.format(w_t), end='')
        else:
            print('\033[0;31;47m', w_t, end='')
    print()

In [57]:
# Print prediction(s)
print('---- Original ----')
print(' '.join(token_to_words(tmp_x[:1][0][:,0],english_tokenizer) ))
print()
translate(simple_rnn_model.predict(tmp_x[:1]), preproc_french_sentences[:1])

---- Original ----
new jersey is sometimes quiet during autumn and it is snowy in april

---- Gold standard ----
new jersey est parfois calme pendant l' automne et il est neigeux en avril

---- Prediction ----
 new jersey est parfois calme en l' et il est est en en


### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

#### Inputs/Outputs shape embedding RNN
- Input shape to Embedding layer (batchsize, input_length ie timesteps). the timesteps = length of the sequences
- Input to the Embedding layer is vector of vocab_size for each token, converted into vector of embed_dim size
- Embeddings output_shape is (None, sequence_length, embbed_dim). This is 3D expected by LSTM !
- output shape LSTM (None, timesteps = sequence_length, num_units). Return_sequence activated to return output at each timestep
- Use of TimeDistributed wrapper over Dense(target_vocab_size) to process each timestep
- for each time step, Dense outputs a probability distribution over target_vocab to predict next word

In [54]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    
    learning_rate = 0.001
    embedding_size = 256
    
    english_input = Input(shape=input_shape[1:], name="input_layer")   # Input shape (,seq_length)
    
    embeddings = Embedding(input_dim = english_vocab_size, output_dim = embedding_size, 
                           input_length= output_sequence_length, name="Embedding_layer")(english_input)
    
    x = LSTM(64, return_sequences=True, activation="tanh", name="LSTM_layer")(embeddings)
    
    preds = TimeDistributed(Dense(french_vocab_size, activation="softmax"), name="Dense_layer")(x)
    
    model = Model(inputs=english_input, outputs=preds, name='Embedding_LSTM')
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

tests.test_embed_model(embed_model)

# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))    # reshaped to (batchsize, seq_length) for Embedding input

# Train the neural network
embed_rnn_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

embed_rnn_model.summary()
    
embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     (None, 21)                0         
_________________________________________________________________
Embedding_layer (Embedding)  (None, 21, 256)           51200     
_________________________________________________________________
LSTM_layer (LSTM)            (None, 21, 64)            82176     
_________________________________________________________________
Dense_layer (TimeDistributed (None, 21, 345)           22425     
Total params: 155,801
Trainable params: 155,801
Non-trainable params: 0
_________________________________________________________________
Train on 110288 samples, validate on 27573 samples
Epoch 1/20
110288/110288 [==============================] - 11s 101us/step - loss: 3.5813 - acc: 0.4027 - val_loss: 2.7548 - val_acc: 0.4216
Epoch 2/20
110288/110288 [==============================] - 10s 87us/step - 

In [55]:
# Print prediction(s)
print('---- Original ----')
print(' '.join(token_to_words(tmp_x[:1][0],english_tokenizer) ))
print()
translate(embed_rnn_model.predict(tmp_x[:1]), preproc_french_sentences[:1])

---- Original ----
new jersey is sometimes quiet during autumn and it is snowy in april

---- Gold standard ----
new jersey est parfois calme pendant l' automne et il est neigeux en avril

---- Prediction ----
 new jersey est parfois calme en l' et il est est neigeux en avril


### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

#### Inputs/Outputs shape Bidirectional RNN
- LSTM or GRU Input shape (batchsize, timesteps, 1). The timesteps is the length of the input sequences (15 for en, 21 for fr)
- output shape Bidir-LSTM (None, timesteps = sequence_length, 2 x num_units) because both outputs are concatenated.
- Return_sequence activated to return output at each timestep
- Use of TimeDistributed wrapper over Dense(target_vocab_size) to process each timestep
- for each time step, Dense outputs a probability distribution over target_vocab to predict next word

In [51]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    
    learning_rate = 0.001
    embedding_size = 256
    
    english_input = Input(shape=input_shape[1:], name="input_layer")  # input to LSTM shape (batch, seq_length, 1)
    
    x = Bidirectional(LSTM(64, return_sequences=True, activation="tanh", name="Bidir_LSTM_layer"), input_shape=input_shape[1:])(english_input)
    
    preds = TimeDistributed(Dense(french_vocab_size, activation="softmax"), name="Dense_layer")(x)
    
    model = Model(inputs=english_input, outputs=preds, name='Bidir_LSTM')
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

tests.test_bd_model(bd_model)

# TODO: Train and Print prediction(s)

# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2],1))  # reshape to (batch, seq_length, 1) for LSTM input

# Train the neural network
bd_rnn_model = bd_model(
                        tmp_x.shape,
                        max_french_sequence_length,
                        english_vocab_size,
                        french_vocab_size)

bd_rnn_model.summary()
    
bd_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        (None, 21, 1)             0         
_________________________________________________________________
bidirectional_7 (Bidirection (None, 21, 128)           33792     
_________________________________________________________________
Dense_layer (TimeDistributed (None, 21, 345)           44505     
Total params: 78,297
Trainable params: 78,297
Non-trainable params: 0
_________________________________________________________________
Train on 110288 samples, validate on 27573 samples
Epoch 1/20
110288/110288 [==============================] - 14s 122us/step - loss: 3.2011 - acc: 0.4503 - val_loss: 2.1757 - val_acc: 0.5025
Epoch 2/20
110288/110288 [==============================] - 12s 108us/step - loss: 1.9134 - acc: 0.5493 - val_loss: 1.7154 - val_acc: 0.5757
Epoch 3/20
110288/110288 [==============================] - 12s 108us

In [53]:
# Print prediction(s)
print('---- Original ----')
print(' '.join(token_to_words(tmp_x[:1][0][:,0],english_tokenizer) ))
print()
translate(bd_rnn_model.predict(tmp_x[:1]), preproc_french_sentences[:1])

---- Original ----
new jersey is sometimes quiet during autumn and it is snowy in april

---- Gold standard ----
new jersey est parfois calme pendant l' automne et il est neigeux en avril

---- Prediction ----
 new jersey est parfois calme en l' et il est est en en


### Model 3 bis: Embedding Bidirectional RNNs (IMPLEMENTATION)

#### Inputs/Outputs shape Embedding Bidirectional RNN
- Embedding requires input shape (batchsize, input_length) and outputs (batchsize, input_length, embedded_dim)
- this shape is OK for input to LSTM (3D)
- LSTM or GRU Input shape (batchsize, timesteps, embedded_dim). Return_sequence activated to return output at each timestep
- output shape Bidir-LSTM (None, timesteps = sequence_length, 2 x LSTM num_units) because both outputs are concatenated.
- Use of TimeDistributed wrapper over Dense(target_vocab_size) to process each timestep
- for each time step, Dense outputs a probability distribution over target_vocab to predict next word

In [16]:
def emb_bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    
    learning_rate = 0.001
    embedding_size = 256
    
    english_input = Input(shape=input_shape[1:], name="input_layer")  # Embedding input (batch, seq_length)
    
    embeddings = Embedding(input_dim = english_vocab_size, output_dim = embedding_size, 
                           input_length= output_sequence_length, name="Embedding_layer")(english_input)
    
    # input shape to LSTM (batchsize, seq_length, embedding_dim) output shape: (batchsize, seq_length, units=64x2)
    x = Bidirectional(LSTM(64, return_sequences=True, activation="tanh"), name="Bidir_LSTM_layer")(embeddings)
    
    preds = TimeDistributed(Dense(french_vocab_size, activation="softmax"), name="Dense_layer")(x)
    
    model = Model(inputs=english_input, outputs=preds, name='Embedding_Bidir_LSTM')
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
   
    return model

#tests.test_bd_model(bd_model)

# TODO: Train and Print prediction(s)

# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))    # reshapped for Embedding input (batch, seq_length)

# Train the neural network
emb_bd_rnn_model = emb_bd_model(
                        tmp_x.shape,
                        max_french_sequence_length,
                        english_vocab_size,
                        french_vocab_size)

emb_bd_rnn_model.summary()

emb_bd_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     (None, 21)                0         
_________________________________________________________________
Embedding_layer (Embedding)  (None, 21, 256)           51200     
_________________________________________________________________
Bidir_LSTM_layer (Bidirectio (None, 21, 128)           164352    
_________________________________________________________________
Dense_layer (TimeDistributed (None, 21, 345)           44505     
Total params: 260,057
Trainable params: 260,057
Non-trainable params: 0
_________________________________________________________________
Train on 110288 samples, validate on 27573 samples
Epoch 1/20
110288/110288 [==============================] - 16s 147us/step - loss: 3.1838 - acc: 0.4493 - val_loss: 2.3086 - val_acc: 0.5024
Epoch 2/20
110288/110288 [==============================] - 15s 140us/step -

In [17]:
# Print prediction(s)
print('---- Original ----')
print(' '.join(token_to_words(tmp_x[:1][0],english_tokenizer) ))
print()
translate(emb_bd_rnn_model.predict(tmp_x[:1]), preproc_french_sentences[:1])

---- Original ----
new jersey is sometimes quiet during autumn and it is snowy in april

---- Gold standard ----
new jersey est parfois calme pendant l' automne et il est neigeux en avril

---- Prediction ----
 new jersey est parfois calme au l' automne il et est neigeux en avril


## Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

### LSTM Encoder-Decoder Architectures using RepeatVector

https://machinelearningmastery.com/encoder-decoder-long-short-term-memory-networks/

https://machinelearningmastery.com/configure-encoder-decoder-model-neural-machine-translation/

- Encoder: One or more LSTM layers can be used to implement the encoder model. The output of this model is a fixed-size vector that represents the internal representation of the input sequence. The number of memory cells in this layer defines the length of this fixed-sized vector.

- Decoder: The decoder must transform the learned internal representation of the input sequence into the correct output sequence. One or more LSTM layers can also be used to implement the decoder model. This model reads from the fixed sized output from the encoder model.

**Issue**: We must connect the encoder to the decoder, and they do not fit. That is, the encoder will produce a 2-dimensional matrix of outputs, where the length is defined by the number of memory cells in the layer. The decoder is an LSTM layer that expects a 3D input of [samples, time steps, features] in order to produce a decoded sequence of some different length defined by the problem.

We can solve this using a `RepeatVector` layer. This layer simply repeats the provided 2D input multiple times to create a 3D output. The RepeatVector layer can be used like an adapter to fit the encoder and decoder parts of the network together. We can configure the RepeatVector to repeat the fixed length vector one time for each time step in the output sequence.

#### Simplified LSTM Encoder Decoder using recommended structure
- https://machinelearningmastery.com/configure-encoder-decoder-model-neural-machine-translation/
- Encoder: Bidirectional (2 units)
- Decoder: 2 units

In [15]:
def encdec_model2(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement
    
    learning_rate = 0.001
    embedding_size = 300
    num_units = 256
    
    ########### ENCODER ###########
    encoder_input = Input(shape=input_shape[1:], name="input_Encoder")
    
    embeddings = Embedding(input_dim = english_vocab_size, output_dim = embedding_size, 
                           input_length= input_shape[1], name="Embedding_layer")(encoder_input)
    
    encoder_output = Bidirectional(LSTM(num_units, return_sequences=False, activation="relu"), name="Bidir_Encoder")(embeddings)
    # output shape of decoder = (batch , num_units)
    
    ########### INTERMEDIARY ###########
    # This adjusts the shape of the Encoder output (2D) to the need of the Decoder (3D input).
    # We repeat the 2D vector over sequence_length times to produce the shape (batchsize, seq_length, LSTM units)
    input_decoder = RepeatVector(output_sequence_length)(encoder_output)
    
    ########### DECODER ###########
    decoder_input = Input(shape=input_decoder.shape[:1], name="input_Decoder")
    
    x = LSTM(num_units, return_sequences=True, activation="relu", name="LSTM_layer1")(input_decoder)
    x = LSTM(num_units, return_sequences=True, activation="relu", name="LSTM_layer2")(x)
    
    preds = TimeDistributed(Dense(french_vocab_size, activation="softmax"), name="Dense_layer")(x)

    ########### END-TO-END MODEL ###########
    model = Model(inputs=encoder_input, outputs=preds, name='Encdec')
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

#tests.test_encdec_model(encdec_model2) # commented out to avoid error: english sequences padded to french sequence max length

# OPTIONAL: Train and Print prediction(s)

# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))   # reshape for embedding input

# Train the neural network
enc_dec_model2 = encdec_model2(
                        tmp_x.shape,
                        max_french_sequence_length,
                        english_vocab_size,
                        french_vocab_size)

enc_dec_model2.summary()

enc_dec_model2.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=15, validation_split=0.2)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_Encoder (InputLayer)   (None, 21)                0         
_________________________________________________________________
Embedding_layer (Embedding)  (None, 21, 300)           59700     
_________________________________________________________________
Bidir_Encoder (Bidirectional (None, 512)               1140736   
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 21, 512)           0         
_________________________________________________________________
LSTM_layer1 (LSTM)           (None, 21, 256)           787456    
_________________________________________________________________
LSTM_layer2 (LSTM)           (None, 21, 256)           525312    
_________________________________________________________________
Dense_layer (TimeDistributed (None, 21, 344)           88408     
Total para

In [16]:
print('---- Original ----')
print(' '.join(token_to_words(tmp_x[:1][0],english_tokenizer) ))
print()
translate(enc_dec_model2.predict(tmp_x[:1]), preproc_french_sentences[:1])

---- Original ----
new jersey is sometimes quiet during autumn and it is snowy in april

---- Gold standard ----
new jersey est parfois calme pendant l' automne et il est neigeux en avril

---- Prediction ----
 new jersey est parfois calme pendant l' et et il est en en avril


### Savings weights of Encoder Decoder model

In [17]:
# Save the weights
enc_dec_model2.save_weights('model_encdec_LSTM')

# Create a new model instance
#model = create_model()

# Restore the weights
#model.load_weights('./saved_model/final_model_3')

# Save the entire model as a SavedModel.
!mkdir -p saved_model
enc_dec_model2.save('saved_model/model_encdec_LSTM_full')

# load saved model
#model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
#model.summary()

### LSTM with higher number of units
- Better results vs shallower version above but requires significantly more training

In [21]:
def encdec_model3(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement
    
    learning_rate = 0.001
    embedding_size = 300
    num_units = 256
    
    ########### ENCODER ###########
    model = Sequential(name='Encdec')
    
    # Embedding layer - Input shape (batch_size, seq_length)
    
    model.add(Embedding(input_dim = english_vocab_size, output_dim = embedding_size, 
                           input_length= input_shape[1], name="Embedding_layer"))
    
    # Bidirectional LSTM : unit shape (batch, seq_length, embedding_size), output shape of decoder = (batch , num_unitsx2)
    model.add(Bidirectional(LSTM(num_units, return_sequences=False, activation="relu"),input_shape=input_shape[1:], name="Bidir_Encoder"))
        
    ########### INTERMEDIARY ###########
    # This adjusts the shape of the Encoder output (2D) to the need of the Decoder (3D input).
    # We repeat the 2D vector over sequence_length times to produce the shape (batchsize, seq_length, LSTM unitsx2)
    model.add(RepeatVector(output_sequence_length))
    
    ########### DECODER ###########
    # Depth 2 LSTM layers input=(batch,seq_len,LSTM_unitsx2) => output shape identical due to return_seq and LSTM units
    model.add(LSTM(num_units*2, return_sequences=True, activation="relu", name="LSTM_layer1"))
    model.add(LSTM(num_units*2, return_sequences=True, activation="relu", name="LSTM_layer2"))
    
    model.add(TimeDistributed(Dense(french_vocab_size, activation="softmax"), name="Dense_layer"))

    ########### END-TO-END MODEL ###########
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

#tests.test_encdec_model(encdec_model3) # commented out to avoid error: english sequences padded to french sequence max length

# OPTIONAL: Train and Print prediction(s)

# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))   # reshape for embedding input

# Train the neural network
enc_dec_model3 = encdec_model3(
                        tmp_x.shape,
                        max_french_sequence_length,
                        english_vocab_size,
                        french_vocab_size)

enc_dec_model3.summary()

enc_dec_model3.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=22, validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Embedding_layer (Embedding)  (None, 21, 300)           59700     
_________________________________________________________________
Bidir_Encoder (Bidirectional (None, 512)               1140736   
_________________________________________________________________
repeat_vector_3 (RepeatVecto (None, 21, 512)           0         
_________________________________________________________________
LSTM_layer1 (LSTM)           (None, 21, 512)           2099200   
_________________________________________________________________
LSTM_layer2 (LSTM)           (None, 21, 512)           2099200   
_________________________________________________________________
Dense_layer (TimeDistributed (None, 21, 344)           176472    
Total params: 5,575,308
Trainable params: 5,575,308
Non-trainable params: 0
_________________________________________________________________


In [22]:
print('---- Original ----')
print(' '.join(token_to_words(tmp_x[:1][0],english_tokenizer) ))
print()
translate(enc_dec_model3.predict(tmp_x[:1]), preproc_french_sentences[:1])

---- Original ----
new jersey is sometimes quiet during autumn and it is snowy in april

---- Gold standard ----
new jersey est parfois calme pendant l' automne et il est neigeux en avril

---- Prediction ----
 new jersey est parfois calme pendant l' automne et il est neigeux en avril


### Savings weights of Encoder Decoder model

In [23]:
# Save the weights
enc_dec_model3.save_weights('final_model_encdec')

# Save the entire model as a SavedModel.
!mkdir -p saved_model
enc_dec_model3.save('saved_model/final_model_encdec_full')

## Encoder Decoder model with GRU RNN
- No bidirectional unit
- 1 layer depth for both encoder and decoder
- Much less parameters compared to LSTM

In [15]:
def encdec_model4(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement
    
    learning_rate = 0.001
    embedding_size = 256
    num_units = 256
    
    model = Sequential(name='enc_dec_model4')
    
    ########### ENCODER ###########
    # Embedding layer - Input shape (batch_size, seq_length)
    model.add(Embedding(input_dim = english_vocab_size, output_dim = embedding_size, 
                           input_length= input_shape[1], name="Embedding_layer"))
    
    # Expected shape for GRU layer input : (batchsize, Seq_length, embedding_size)
    model.add(GRU(num_units, go_backwards=True, name='GRU_encoder'))
    # output shape of decoder = (batch , num_units)
    
    ########### INTERMEDIARY ###########
    # This adjusts the shape of the Encoder output (2D) to the need of the Decoder (3D input).
    # We repeat the 2D vector over sequence_length times to produce the shape (batchsize, seq_length, num_units)
    model.add(RepeatVector(output_sequence_length, name='Glue'))
    # output shape of glue intermediary step = (batch , seq_length, num_units) compatible with expected input for Decoder GRU  
    
    ########### DECODER ###########
       
    model.add(GRU(num_units, return_sequences=True, name='GRU_decoder'))
    
    model.add(TimeDistributed(Dense(french_vocab_size), name='Dense'))
    
    model.add(Activation('softmax', name='softmax'))
    # output shape of decoder = (batch , french_vocab_size) probability distribution over target vocabulary
    
    ########### END-TO-END MODEL ###########
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

#tests.test_encdec_model(encdec_model) # commented out to avoid error: english sequences padded to french sequence max length

# OPTIONAL: Train and Print prediction(s)

# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length) # pad input sequence to output sequence length
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))    # reshape for embedding input

# Train the neural network
enc_dec_model4 = encdec_model4(
                        tmp_x.shape,
                        max_french_sequence_length,
                        english_vocab_size,
                        french_vocab_size)

enc_dec_model4.summary()

enc_dec_model4.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=30, validation_split=0.2)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Embedding_layer (Embedding)  (None, 21, 256)           50944     
_________________________________________________________________
GRU_encoder (GRU)            (None, 256)               393984    
_________________________________________________________________
Glue (RepeatVector)          (None, 21, 256)           0         
_________________________________________________________________
GRU_decoder (GRU)            (None, 21, 256)           393984    
_________________________________________________________________
Dense (TimeDistributed)      (None, 21, 344)           88408     
_________________________________________________________________
softmax (Activation)         (None, 21, 344)           0         
Total params: 927,320
Trainable params: 927,320
Non-trainable params: 0
_________________________________________________________________
Trai

In [16]:
print('---- Original ----')
print(' '.join(token_to_words(tmp_x[:1][0],english_tokenizer) ))
print()
translate(enc_dec_model4.predict(tmp_x[:1]), preproc_french_sentences[:1])

---- Original ----
new jersey is sometimes quiet during autumn and it is snowy in april

---- Gold standard ----
new jersey est parfois calme pendant l' automne et il est neigeux en avril

---- Prediction ----
 new jersey est parfois calme pendant l' automne et il est neigeux en avril


In [17]:
# Save the weights
enc_dec_model4.save_weights('final_model_encdec4')

# Save the entire model as a SavedModel.
!mkdir -p saved_model
enc_dec_model4.save('saved_model/final_model_encdec4_full')

### More complex Encoder-Decoder with GRU RNNs
- Bidirectional unit for the encoder (2 layers)
- 2 layers depth for the decoder
- Good convergence and accuracy, at par with LSTM, with less parameters

In [26]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement
    
    learning_rate = 0.001
    embedding_size = 512
    num_units = 256
    
    ########### ENCODER ###########
    # Expected shape for Embedding layer input : (batchsize, Seq_length)
    encoder_input = Input(shape=input_shape[1:], name="input_Encoder")  
    
    embeddings = Embedding(input_dim = english_vocab_size, output_dim = embedding_size, 
                           input_length= input_shape[1], name="Embedding_layer")(encoder_input)
    # output shape of embeddings = (batch , seq_length, embedding_size)
    
    encoder_output = Bidirectional(GRU(num_units, return_sequences=False, activation="tanh"), name="BiGRU_encoder")(embeddings)
    # output shape of decoder = (batch , num_units)
    
    ########### INTERMEDIARY ###########
    # This adjusts the shape of the Encoder output (2D) to the need of the Decoder (3D input).
    # We repeat the 2D vector over sequence_length times to produce the shape (batchsize, seq_length, num_units)
    sequenced_context_vector = RepeatVector(output_sequence_length)(encoder_output)
    # output shape of glue intermediary step = (batch , seq_length, num_units) compatible with expected input for Decoder GRU  
    
    ########### DECODER ###########
       
    x = GRU(num_units, return_sequences=True, activation="tanh", name="GRU_layer1")(sequenced_context_vector)
    # output shape of GRU = (batch , seq_length, num_units) because return_seq activated
    x = GRU(num_units, return_sequences=True, activation="tanh", name="GRU_layer2")(x)
    # output shape of GRU = (batch , seq_length, num_units) because return_seq activated
    
    preds = TimeDistributed(Dense(french_vocab_size, activation="softmax"), name="Dense_layer")(x)
    # output shape of decoder = (batch , french_vocab_size) probability distribution over target vocabulary
    
    ########### END-TO-END MODEL ###########
    model = Model(inputs=encoder_input, outputs=preds, name='Encdec')
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

#tests.test_encdec_model(encdec_model) # commented out to avoid error: english sequences padded to french sequence max length

# OPTIONAL: Train and Print prediction(s)

# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length) # pad input sequence to output sequence length
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))    # reshape for embedding input

# Train the neural network
enc_dec_model = encdec_model(
                        tmp_x.shape,
                        max_french_sequence_length,
                        english_vocab_size,
                        french_vocab_size)

enc_dec_model.summary()

enc_dec_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=25, validation_split=0.2)


Model: "Encdec"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_Encoder (InputLayer)   (None, 21)                0         
_________________________________________________________________
Embedding_layer (Embedding)  (None, 21, 512)           101888    
_________________________________________________________________
BiGRU_encoder (Bidirectional (None, 512)               1181184   
_________________________________________________________________
repeat_vector_8 (RepeatVecto (None, 21, 512)           0         
_________________________________________________________________
GRU_layer1 (GRU)             (None, 21, 256)           590592    
_________________________________________________________________
GRU_layer2 (GRU)             (None, 21, 256)           393984    
_________________________________________________________________
Dense_layer (TimeDistributed (None, 21, 345)           88665

C:\Users\lveys\anaconda3\envs\nlp_py37\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 110288 samples, validate on 27573 samples
Epoch 1/25
110288/110288 [==============================] - 23s 210us/step - loss: 2.7213 - accuracy: 0.4575 - val_loss: nan - val_accuracy: 0.5223
Epoch 2/25
110288/110288 [==============================] - 22s 202us/step - loss: 1.7588 - accuracy: 0.5601 - val_loss: nan - val_accuracy: 0.5959
Epoch 3/25
110288/110288 [==============================] - 22s 203us/step - loss: 1.4468 - accuracy: 0.6125 - val_loss: nan - val_accuracy: 0.6296
Epoch 4/25
110288/110288 [==============================] - 24s 214us/step - loss: 1.2990 - accuracy: 0.6430 - val_loss: nan - val_accuracy: 0.6648
Epoch 5/25
110288/110288 [==============================] - 24s 217us/step - loss: 1.1607 - accuracy: 0.6799 - val_loss: nan - val_accuracy: 0.6497
Epoch 6/25
110288/110288 [==============================] - 24s 214us/step - loss: 1.0459 - accuracy: 0.7029 - val_loss: nan - val_accuracy: 0.7118
Epoch 7/25
110288/110288 [==============================] - 2

In [27]:
# Print prediction(s)
print('---- Original ----')
print(' '.join(token_to_words(tmp_x[:1][0],english_tokenizer) ))
print()
translate(enc_dec_model.predict(tmp_x[:1]), preproc_french_sentences[:1])

---- Original ----
new jersey is sometimes quiet during autumn and it is snowy in april

---- Gold standard ----
new jersey est parfois calme pendant l' automne et il est neigeux en avril

---- Prediction ----
 new jersey est parfois calme pendant l' automne et il est neigeux en avril


### Savings weights of Encoder Decoder model

In [29]:
# Save the weights
enc_dec_model.save_weights('final_model_encdec_GRU')

# Save the entire model as a SavedModel.
#!mkdir -p saved_model
enc_dec_model.save('saved_model/final_model_encdec_GRU_full')

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

#### Inputs/Outputs shape Customl RNN
- Embedding requires input shape (batchsize, input_length) and outputs (batchsize, input_length, embedded_dim)
- this shape is OK for input to LSTM (3D)
- LSTM or GRU Input shape (batchsize, timesteps, embedded_dim). Return_sequence activated to return output at each timestep
- output shape Bidir-LSTM (None, timesteps = sequence_length, 2 x LSTM num_units) because both outputs are concatenated.
- Use of TimeDistributed wrapper over Dense(target_vocab_size) to process each timestep
- for each time step, Dense outputs a probability distribution over target_vocab to predict next word


In [48]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    
    learning_rate = 6e-3
    embedding_size = 256
    units = 256
    
    model = Sequential()
    
    ########### ENCODER ###########
    
    model.add(Embedding(input_dim = english_vocab_size, output_dim = embedding_size, 
                           input_length= input_shape[1], name="Embedding_layer"))
    model.add(Bidirectional(LSTM(units, return_sequences=False), name='Bi_LSTM_encoder'))
    
    ########### INTERMEDIARY ###########
    
    model.add(RepeatVector(output_sequence_length, name='Glue'))
    
    ########### DECODER ###########
    
    model.add(LSTM(units, return_sequences=True, name='LSTM_decoder'))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax'), name='Dense'))
    
    ########### END-TO-END MODEL ###########
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

tests.test_model_final(model_final)

# TODO: Train the final model

tmp_x = pad(preproc_english_sentences, max_french_sequence_length) # pad input sequence to output sequence length
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))    # reshape for embedding input

# Train the neural network
final_model = model_final(
                        tmp_x.shape,
                        max_french_sequence_length,
                        english_vocab_size,
                        french_vocab_size)

final_model.summary()

final_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)

print('Final Model Loaded')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Embedding_layer (Embedding)  (None, 21, 256)           50944     
_________________________________________________________________
Bi_LSTM_encoder (Bidirection (None, 512)               1050624   
_________________________________________________________________
Glue (RepeatVector)          (None, 21, 512)           0         
_________________________________________________________________
LSTM_decoder (LSTM)          (None, 21, 256)           787456    
_________________________________________________________________
Dense (TimeDistributed)      (None, 21, 344)           88408     
Total params: 1,977,432
Trainable params: 1,977,432
Non-trainable params: 0
_________________________________________________________________
Train on 110288 samples, validate on 27573 samples
Epoch 1/20
110288/110288 [==============================] - 45s 408us/step - loss

In [49]:
print('---- Original ----')
print(' '.join(token_to_words(tmp_x[:1][0],english_tokenizer) ))
print()
translate(final_model.predict(tmp_x[:1]), preproc_french_sentences[:1])

---- Original ----
new jersey is sometimes quiet during autumn and it is snowy in april

---- Gold standard ----
new jersey est parfois calme pendant l' automne et il est neigeux en avril

---- Prediction ----
 new jersey est parfois calme pendant l' automne et il est neigeux en avril


### Save Best Model

In [50]:
# Save the weights
final_model.save_weights('final_model')

In [51]:
# Save the entire model as a SavedModel.
!mkdir -p saved_model
final_model.save('saved_model/final_model_full')

## Prediction (IMPLEMENTATION)
- the objective is to reach a minimum of 95% validation accuracy so that to have meaningful predictions

In [34]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # align padded english sequence to the length of french padded length
    x = pad_sequences(x)     
    
    # Instantiate the best model
    model = model_final(
                        x.shape,
                        y.shape[1],
                        len(x_tk.word_index),
                        len(y_tk.word_index))
    
    model.summary()
                                 
    # train the model - Reach a minimum of 94% accuracy so that to have meaningful predictions
    model.fit(x, y, batch_size=1024, epochs=10, validation_split=0.2)

    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))

final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Embedding_layer (Embedding)  (None, 15, 256)           50944     
_________________________________________________________________
Bi_LSTM_encoder (Bidirection (None, 512)               1050624   
_________________________________________________________________
Glue (RepeatVector)          (None, 21, 512)           0         
_________________________________________________________________
LSTM_decoder (LSTM)          (None, 21, 256)           787456    
_________________________________________________________________
Dense (TimeDistributed)      (None, 21, 344)           88408     
Total params: 1,977,432
Trainable params: 1,977,432
Non-trainable params: 0
_________________________________________________________________
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 37s 332us/step - loss

## ALTERNATIVES
- Encoder-Decoder model based on GRU cells with depth of 2 layers for both RNN

In [54]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    
    # align padded english sequence to the length of french padded length
    x = pad_sequences(x)     
    
    # Instantiate the best model
    model = encdec_model(
                        x.shape,
                        y.shape[1],
                        len(x_tk.word_index),
                        len(y_tk.word_index))
    
    model.summary()
                                 
    # train the model - Reach a minimum of 94% accuracy so that to have meaningful predictions
    model.fit(x, y, batch_size=1024, epochs=20, validation_split=0.2)

    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_Encoder (InputLayer)   (None, 15)                0         
_________________________________________________________________
Embedding_layer (Embedding)  (None, 15, 512)           101888    
_________________________________________________________________
Bidir_encoder (Bidirectional (None, 512)               1181184   
_________________________________________________________________
repeat_vector_4 (RepeatVecto (None, 21, 512)           0         
_________________________________________________________________
LSTM_layer1 (GRU)            (None, 21, 256)           590592    
_________________________________________________________________
LSTM_layer2 (GRU)            (None, 21, 256)           393984    
_________________________________________________________________
Dense_layer (TimeDistributed (None, 21, 344)           88408     
Total para

## OPTION TO LOAD PRE-EXISTING MODEL WEIGHTS ######

In [34]:
def final_predictions_(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    
    # align padded english sequence to the length of french padded length
    x = pad_sequences(x, y.shape[1])    
    
    # Instantiate the best model
    model = encdec_model(
                        x.shape,
                        y.shape[1],
                        len(x_tk.word_index),
                        len(y_tk.word_index))
    from keras.models import load_model
    load_model('saved_model/final_model_encdec_GRU_full')
    #model.load_weights('final_model_encdec_GRU')
    
    model.summary()
    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions_(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

C:\Users\lveys\anaconda3\envs\nlp_py37\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Model: "Encdec"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_Encoder (InputLayer)   (None, 21)                0         
_________________________________________________________________
Embedding_layer (Embedding)  (None, 21, 512)           101888    
_________________________________________________________________
BiGRU_encoder (Bidirectional (None, 512)               1181184   
_________________________________________________________________
repeat_vector_13 (RepeatVect (None, 21, 512)           0         
_________________________________________________________________
GRU_layer1 (GRU)             (None, 21, 256)           590592    
_________________________________________________________________
GRU_layer2 (GRU)             (None, 21, 256)           393984    
_________________________________________________________________
Dense_layer (TimeDistributed (None, 21, 345)           88665

## Submission
When you're ready to submit, complete the following steps:
1. Review the [rubric](https://review.udacity.com/#!/rubrics/1004/view) to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

### Generate the html

**Save your notebook before running the next cell to generate the HTML output.** Then submit your project.

In [56]:
# Save before you run this cell!
!!jupyter nbconvert *.ipynb

['[NbConvertApp] Converting notebook machine_translation-Copy1.ipynb to html',
 '[NbConvertApp] Writing 509712 bytes to machine_translation-Copy1.html',
 '[NbConvertApp] Converting notebook machine_translation.ipynb to html',
 '[NbConvertApp] Writing 425160 bytes to machine_translation.html']

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?